# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
import os
import sys
import datetime

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('flopy version: {}'.format(flopy.__version__))

flopy is installed in /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/flopy
3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 14:38:56) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
flopy version: 3.3.1


Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("..", "data", "freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False)

We can see the ``Modelgrid`` instance has generic entries, as does ``start_datetime``

In [3]:
ml.modelgrid

xll:622241.1904510253; yll:3343617.741737109; rotation:15.0; proj4_str:+proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs; units:meters; lenuni:2

In [4]:
ml.modeltime.start_datetime

'1/1/2015'

Setting the attributes of the ``ml.modelgrid`` is easy:

In [5]:
proj4_str = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
ml.modelgrid.set_coord_info(xoff=123456.7, yoff=765432.1, angrot=15.0, proj4=proj4_str)
ml.dis.start_datetime = '7/4/1776'

In [6]:
ml.modeltime.start_datetime

'7/4/1776'

### Some netCDF export capabilities:

#### Export the whole model (inputs and outputs)

In [7]:
# make directory
pth = os.path.join('data', 'netCDF_export')
if not os.path.exists(pth):
    os.makedirs(pth)

In [8]:
fnc = ml.export(os.path.join(pth, ml.name+'.in.nc'))
hds = flopy.utils.HeadFile(os.path.join(model_ws,"freyberg.hds"))
flopy.export.utils.output_helper(os.path.join(pth, ml.name+'.out.nc'), ml, {"hds":hds})

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80
initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80


#### export a single array to netcdf or shapefile

In [9]:
# export a 2d array
ml.dis.top.export(os.path.join(pth, 'top.nc'))
ml.dis.top.export(os.path.join(pth, 'top.shp'))

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80
wrote data/netCDF_export/top.shp


#### sparse export of stress period data for a boundary condition package  
* excludes cells that aren't in the package (aren't in `package.stress_period_data`)  
* by default, stress periods with duplicate parameter values (e.g., stage, conductance, etc.) are omitted
(`squeeze=True`); only stress periods with different values are exported  
* argue `squeeze=False` to export all stress periods

In [10]:
ml.drn.stress_period_data.export(os.path.join(pth, 'drn.shp'), sparse=True)

wrote data/netCDF_export/drn.shp


#### Export a 3d array

In [11]:
#export a 3d array
ml.upw.hk.export(os.path.join(pth, 'hk.nc'))
ml.upw.hk.export(os.path.join(pth, 'hk.shp'))

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80
wrote data/netCDF_export/hk.shp


#### Export a number of things to the same netCDF file

In [12]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(pth, 'test.nc'))
ml.upw.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80


### Export whole packages to a netCDF file

In [13]:
# export mflist
fnc = ml.wel.export(os.path.join(pth, 'packages.nc'))
ml.upw.export(fnc)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-06-26T13:32:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), 

### Export the whole model to a netCDF

In [14]:
fnc = ml.export(os.path.join(pth, 'model.nc'))
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-06-26T13:32:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), 

## Export output to netcdf

FloPy has utilities to export model outputs to a netcdf file. Valid output types for export are MODFLOW binary head files, formatted head files, cell budget files, seawat concentration files, and zonebudget output.

Let's use output from the Freyberg model as an example of these functions

In [15]:
# load binary head and cell budget files
fhead = os.path.join(model_ws, 'freyberg.hds')
fcbc = os.path.join(model_ws, 'freyberg.cbc')

hds = flopy.utils.HeadFile(fhead)
cbc = flopy.utils.CellBudgetFile(fcbc)

export_dict = {"hds": hds,
               "cbc": cbc}

# export head and cell budget outputs to netcdf
fnc = flopy.export.utils.output_helper(os.path.join(pth, "output.nc"), ml, export_dict)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80
error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-06-26T13:33:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), 

### Exporting zonebudget output

zonebudget output can be exported with other modflow outputs, and is placed in a seperate group which allows the user to post-process the zonebudget output before exporting.

Here are two examples on how to export zonebudget output with a binary head and cell budget file

__Example 1__: No postprocessing of the zonebudget output

In [16]:
# load the zonebudget output file
zonbud_ws = os.path.join("..", "data", "zonbud_examples")
fzonbud = os.path.join(zonbud_ws, "freyberg_mlt.2.csv")
zon_arrays = flopy.utils.zonbud.read_zbarray(os.path.join(zonbud_ws, "zonef_mlt.zbr"))

zbout = flopy.utils.ZoneBudgetOutput(fzonbud, ml.dis, zon_arrays)
zbout

ZoneBudgetOutput Class
----------------------

Number of zones: 4
Unique zones: 0, 1, 2, 3
Number of buget records: 3291

In [17]:
export_dict = {'hds': hds,
               'cbc': cbc}

fnc = flopy.export.utils.output_helper(os.path.join(pth, "output_with_zonebudget.nc"),
                                       ml, export_dict)

fnc = zbout.export(fnc, ml)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80
error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-06-26T13:33:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), 

A budget_zones variable has been added to the root group and a new zonebudget group has been added to the netcdf file which hosts all of the budget data

__Example 2__: postprocessing zonebudget output then exporting

In [18]:
# load the zonebudget output and get the budget information
zbout = flopy.utils.ZoneBudgetOutput(fzonbud, ml.dis, zon_arrays)
df = zbout.dataframe
df

,kper,kstp,zone,storage,constant head,other zones,total,zone 0,zone 1,zone 2,zone 3,tslen,totim
0,0,0,1,0.000000,-821.281900,-1570.821,-2392.103,0.0,0.0000,-1530.422,-40.3993,1.0,1.0
1,0,0,2,0.000000,-648.804700,630.730,-18.075,0.0,1530.4220,0.000,-899.6920,1.0,1.0
2,0,0,3,0.000000,-976.232200,940.092,-36.140,0.0,40.3993,899.692,0.0000,1.0,1.0
3,1,0,1,218.568500,-816.347300,-1173.221,-1770.999,0.0,0.0000,-1134.937,-38.2835,1.0,2.0
4,1,0,2,191.816342,-643.938700,433.628,-18.493,0.0,1134.9370,0.000,-701.3090,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3286,1095,0,2,-626.408120,-505.116270,1113.766,-17.758,0.0,2489.4040,0.000,-1375.6380,1.0,1096.0
3287,1095,0,3,-627.235750,-801.732376,1393.454,-35.514,0.0,17.8163,1375.638,0.0000,1.0,1096.0
3288,1096,0,1,0.000000,-230.548300,-152.236,-382.784,0.0,0.0000,-205.822,53.5856,1.0,1097.0
3289,1096,0,2,0.000000,15.864900,-30.796,-14.931,0.0,205.8220,0.000,-236.6170,1.0,1097.0


Let's calculate a yearly volumetric budget from the zonebudget data

In [19]:
# get a dataframe of volumetric budget information
vol_df = zbout.volumetric_flux()

# add a year field to the dataframe using datetime
start_date = ml.modeltime.start_datetime
start_date = datetime.datetime.strptime(start_date, "%m/%d/%Y")
nzones = len(zbout.zones) - 1

year = [start_date.year] * nzones
for totim in vol_df.totim.values[:-nzones]:
    t = start_date + datetime.timedelta(days=totim)
    year.append(t.year)

vol_df['year'] = year
    
# calculate yearly volumetric change using pandas
totim_df = vol_df.groupby(['year', 'zone'], as_index=False)['totim'].max()
yearly = vol_df.groupby(['year', 'zone'], as_index=False)[['storage', 'constant head', 'other zones',
                                                           'zone 1', 'zone 2', 'zone 3']].sum()
yearly['totim'] = totim_df['totim']
yearly

,year,zone,storage,constant head,other zones,zone 1,zone 2,zone 3,totim
0,1776,1,81203.267170,-134930.451200,-176631.7910,0.0000,-172310.4031,-4321.3803,181.0
1,1776,2,37268.485533,-102758.917473,62223.7540,172310.4031,0.0000,-110086.6556,181.0
2,1776,3,37296.183058,-158237.438437,114408.0385,4321.3803,110086.6556,0.0000,181.0
3,1777,1,91777.777222,-265545.355300,-407281.7640,0.0000,-398769.3120,-8512.4548,546.0
4,1777,2,42820.852999,-200142.961123,150829.2320,398769.3120,0.0000,-247940.0845,546.0
5,1777,3,42818.153555,-312276.958409,256452.5519,8512.4548,247940.0845,0.0000,546.0
6,1778,1,108226.799462,-255186.105500,-378440.1290,0.0000,-371720.4820,-6719.6672,911.0
7,1778,2,44930.564942,-188836.557151,137315.0100,371720.4820,0.0000,-234405.4742,911.0
8,1778,3,44936.970500,-299267.807443,241125.1428,6719.6672,234405.4742,0.0000,911.0
9,1779,1,5348.709896,-129448.085100,-231291.1680,0.0000,-227734.8190,-3556.3570,1097.0


And finally, export the pandas dataframe to netcdf

In [20]:
# process the new dataframe into a format that is compatible with netcdf exporting
zbncf = zbout.dataframe_to_netcdf_fmt(yearly, flux=False)

# export to netcdf
export_dict = {"hds": hds,
               "cbc": cbc,
               "zbud": zbncf}

fnc = flopy.export.utils.output_helper(os.path.join(pth, "output_with_zonebudget.2.nc"),
                                       ml, export_dict)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +type=crs
initialize_geometry::nc_crs = epsg:4326
transforming coordinates using = proj=noop ellps=GRS80
error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-06-26T13:33:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), 